In [ ]:
# Setting up a custom stylesheet in IJulia
file = open("style.css") # A .css file in the same folder as this notebook file
styl = read(file, String) # Read the file
HTML("$styl") # Output as HTML

# Functions in Julia I

<h2>In this lecture</h2>

- [Outcome](#Outcome)
- [Functions in Julia](#Functions-in-Julia)
- [Command line help](#Command-line-help)
- [Built-in mathematical functions](#Built-in-mathematical-functions)
- [Multiple dispatch](#Multiple-dispatch)


<hr>
<h2>Outcome</h2>

After this lecture, you will be able to: 

- Say how a Julia function works
- Describe the difference between built-in and user-defined functions in Julia
- List and use some basic mathematical functions
- Describe Julia's multiple dispatch using ``muladd()`` as the example

[Back to the top](#In-this-lecture)

<h2>Functions in Julia</h2>

Recall a recent example using the funtion ``println()``:

In [ ]:
greeting = "Hello, world!!"
println(greeting) 

This is a good example of the basics of how a function works: it receives some input (in this case, the value of a variable) and makes something happen (in this case, showing a string on the screen).

Many a function generates a value. In fact, possibly the most common use of functions is to create values to be assigned to variables:

In [1]:
a, b, c = cos(0.2), log(10), abs(-1.22)  # multiple assignment to three separate variables

(0.9800665778412416, 2.302585092994046, 1.22)

In [2]:
a

0.9800665778412416

Julia is a functional language, which means that in a sense the basic unit for organising code in Julia is the function. For instance, ``typeof()`` is a function that provides information which in other types of languages is obtained via other means. If you read the Julia code (it is publicly available, see www.julialang.org), you will see that virtually everything there is done via functions.

This means of course that when you write code, you should also as far as possible organise your code in funtions. We address user-defined functions in Lecture 9 (the one after this one).

[Back to the top](#In-this-lecture)

<h2>Command line help</h2>

To get help in an IJulia, type "?" followed by a topic. For functions, you'll get the syntax (there may be several versions) and in other cases some brief guidance. A few example queries follow, concerning ```cos```, ```help``` and
```floor```

In [ ]:
? cos # a standard maths function gives what you expect, with bonus for explaining what happens to arrays


In [ ]:
?help   # space or not after after "?" makes no difference (but hopping to a new line does!)

Ah well, asking for help in general isn't that helpful ... the right particular question gets good help, but they can be hard to find.

In [3]:
? floor

search: floor OverflowError StackOverflowError



```
floor([T,] x)
floor(x; digits::Integer= [, base = 10])
floor(x; sigdigits::Integer= [, base = 10])
```

`floor(x)` returns the nearest integral value of the same type as `x` that is less than or equal to `x`.

`floor(T, x)` converts the result to type `T`, throwing an `InexactError` if the value is not representable.

`digits`, `sigdigits` and `base` work as for [`round`](@ref).

---

```
floor(x::Period, precision::T) where T <: Union{TimePeriod, Week, Day} -> T
```

Round `x` down to the nearest multiple of `precision`. If `x` and `precision` are different subtypes of `Period`, the return value will have the same type as `precision`.

For convenience, `precision` may be a type instead of a value: `floor(x, Dates.Hour)` is a shortcut for `floor(x, Dates.Hour(1))`.

```jldoctest
julia> floor(Dates.Day(16), Dates.Week)
2 weeks

julia> floor(Dates.Minute(44), Dates.Minute(15))
30 minutes

julia> floor(Dates.Hour(36), Dates.Day)
1 day
```

Rounding to a `precision` of `Month`s or `Year`s is not supported, as these `Period`s are of inconsistent length.

---

```
floor(dt::TimeType, p::Period) -> TimeType
```

Return the nearest `Date` or `DateTime` less than or equal to `dt` at resolution `p`.

For convenience, `p` may be a type instead of a value: `floor(dt, Dates.Hour)` is a shortcut for `floor(dt, Dates.Hour(1))`.

```jldoctest
julia> floor(Date(1985, 8, 16), Dates.Month)
1985-08-01

julia> floor(DateTime(2013, 2, 13, 0, 31, 20), Dates.Minute(15))
2013-02-13T00:30:00

julia> floor(DateTime(2016, 8, 6, 12, 0, 0), Dates.Day)
2016-08-06T00:00:00
```


... while in this case the help may be a bit too detailed! It does alert you to the variety of outputs that ```floor``` can give you. Which one you get depends on the type signature of the variables you provide, and this is your first sight of one of Julia's greatest features: multiple dispatch. More on this feature below, though on this course you only glimpse its power (it is at the heart of much that makes Julia such a wonderful language).

**This rapid, concise help is a wonderful resource to have on hand. USE IT OFTEN,    especially for spelling and syntax.**

[Back to the top](#In-this-lecture)

<h2>Built-in mathematical functions</h2>

Julia has many built-in functions, and for scientific and technical computing the most important of these are mathematical. The following lists a very small part of what is available:

``exp()``

``log()``    ...  NB! this is the *natural* logarithm

``log10()``  ...    *this* is the logarithm to base 10


``cos()``    ... argument must be in radians

``sin()``

``tan()``

``acos()``   ... the result is a value in radians

``asin()``

``atan(x)``   ...  NB! this only returns some angles (between -π/2 and π/2, to be exact)

``atan(y,x)``  ... The *atan2* of many languages (including Python and Matlab); use this to get angles from quadrants two and three; it requires two inputs


``floor()``

``ceil()``

``rem()``

``round()``  ... use round(Int, x) to convert any x of abstract type Real to an integer type (usually Int64)  

**DO EXPLORE THESE USING THE "?" QUERY METHOD!**

And if you enter the first one or more letters of a function, and press Tab, you'll see a list of possible completions. This includes all the available function names that start with that letter or letter. This is a good way to search for function names. For example:

In [ ]:
d      # Tab to show completions, use up and down arrows to scroll the list in box

[Back to the top](#In-this-lecture)

<h2>Multiple dispatch</h2>

This is related to the fact that many functions allow, or even require, more than one input value. For example, ``binomial`` requires two values, and ``muladd()`` requires three (look them up!).

Moreover, functions must accept inputs of more than one type. For instance, if we enter ``cos(1)`` then we are sending a value with type ``Int64`` to Julia's ``cos()`` function. And if we enter "``cos(1.)`` we are sending a value of type ``Float64``. But the value created by Julia's ``cos()`` function in both cases is 0.5403023058681398.

Which is exactly what we want, of course. BUT!

A function name points to a code body. When the code actually executes, it should be specialised on one type, else it will run very slowly, if at all. A function such as ``cos()`` actually has several such code bodies, to deal with inputs of different types. These are called the *methods* of that function. So how do Julia functions manage to have only one name, but many methods?

The answer is multiple dispatch: Julia allows many methods, as long as they differ according to input type. All the input values are used to check this, hence the name multiple dispatch. The pattern of types in the input values in a given function call is its *type signature*. For example, ``muladd()`` has 12 permissible type signatures, which we see by using the function methods():

In [ ]:
methods(muladd)

Many of the bugs you can expect to see if you program in Julia arise because a function was called with values for which there is no method. That is, the type signature of the input values was wrong. You solve this in one of two ways
- by making sure the input values have the right type signature, or 
- by writing an additional method for the required type signature

[Back to the top](#In-this-lecture)